In [ ]:
######
#tool -no_amf_align -buffer 1000 -int 1 12345 -h reference.hap.gz -l reference.legend.gz -m genetic_map -g phasing.haps.gz -o_gz -o chr1_1_12345
import os,glob

wdir = "05.imputation/"
inDir = wdir + "INPUTs/"
outDir = wdir + "OUTPUTs/"
shDir = wdir + "SCRIPTs/01.imputationsh/"
phasingDir = "04.phasing/OUTPUTs/03.5Ksplit/"
tool = "TOOLs/impute4.1.2_r300.2"
refDir = inDir + "KGP3KRG/"
mapDir = inDir + "map/"

def mk_sh(samples,chr,front,tail,ifront,itail):
	print("mk_sh : ....%schr_%s_%s  to  %s_%s"%(chr,front,tail,ifront,itail))
	legend =  refDir+"%s_%s_%s.legend.gz"%(chr,front,tail)
	hap = refDir + "%s_%s_%s.hap.gz"%(chr,front,tail) 
	map = mapDir +"genetic_map_%s_combined_b37.txt"%(chr)
	for sample in samples:
		input = phasingDir + "JG.phasing.%s.%s.haps.gz"%(chr,sample)
		output = outDir + "JG.imputation.%s.%s.%s_%s"%(chr,sample,ifront,itail)
		with open(shDir+"Phased.to.imputation"+chr+"."+sample+"."+ifront+"_"+itail+".sh",'w') as shwrite:
			shwrite.write("%s -no_maf_align -buffer 1000 -int %s %s -h %s -l %s -m %s -g %s -o_gz -o %s" \
                          %(tool,ifront,itail,hap,legend,map,input,output))

def main():
	print("main : ...")
	samples = glob.glob("04.phasing/INPUTs/5KsplitSample/*.sampleID")
	sample_index = [s.replace("04.phasing/INPUTs/5KsplitSample/","") for s in samples]

	ref_file = open(inDir + "imputation.POS.auto.txt","r")
	ref_list = [x.replace('\n','').split('\t') for x in ref_file]

	for ref_pos,imputation_pos in ref_list[1:]:
		chr,front,tail = ref_pos.split('_')
		ichr,ifront,itail = imputation_pos.split('_')
		mk_sh(sample_index,chr,front,tail,ifront,itail)


main()



TOOLs/impute4.1.2_r300.2 -no_maf_align -buffer 1000 -int 96000001 101000000 \
-h 05.imputation/INPUTs/KGP3KRG/chr6_96000001_101000000.hap.gz \
-l 05.imputation/INPUTs/KGP3KRG/chr6_96000001_101000000.legend.gz \
-m 05.imputation/INPUTs/map/genetic_map_chr6_combined_b37.txt \
-g 04.phasing/OUTPUTs/03.5Ksplit/JG.phasing.chr6.75001_77469.sampleID.haps.gz 
-o_gz -o 05.imputation/OUTPUTs/JG.imputation.chr6.75001_77469.sampleID.96000001_101000000















######
#tool -no_amf_align -buffer 1000 -int 1 12345 -h reference.hap.gz -l reference.legend.gz -m genetic_map -g phasing.haps.gz -o_gz -o chr1_1_12345

import glob,os
Dir = "05.imputation/"
inDir = Dir + "INPUTs/"
outDir = Dir + "OUTPUTs/"
shDir = Dir + "SCRIPTs/02.mergeGen/"
phasingDir = "04.phasing/OUTPUTs/03.5Ksplit/"
refDir = inDir + "KGP3KRG/"
mapDir = inDir + "map/"
tool ="TOOLs/qctool"

def mk_sh(samples,chr,front,tail,ifront,itail):
        with open(shDir+"JG.imputed.%s.%s_%s.sampleMerge.sh"%(chr,ifront,itail),'w') as shwrite:
                shwrite.write("%s "%(tool))
                for sample in samples:
                        sample_input = phasingDir+"JG.phasing.%s.%s.sample"%(chr,sample)
                        gen_input = outDir + "01.imputation/JG.imputation.%s.%s.%s_%s.gen.gz"%(chr,sample,ifront,itail)
                        shwrite.write(" -g %s -s %s"%(gen_input,sample_input))
                output = outDir + "02.mergeGen/JG.imputation.mergeGen.%s.%s_%s.gen.gz"%(chr,ifront,itail)
                outsample = outDir + "02.mergeGen/JG.imputation.mergeGen.%s.%s_%s.sample"%(chr,ifront,itail)
                shwrite.write(" -og %s -os %s\n"%(output,outsample))

def main():
        print("main : ...")
        samples = glob.glob("04.phasing/INPUTs/5KsplitSample/*.sampleID")
        sample_index = [s.replace("04.phasing/INPUTs/5KsplitSample/","") for s in samples]

        ref_file = open(inDir + "imputation.POS.auto.txt","r")
        ref_list = [x.replace('\n','').split('\t') for x in ref_file]

        for ref_pos,imputation_pos in ref_list[1:]:
                chr,front,tail = ref_pos.split('_')
                ichr,ifront,itail = imputation_pos.split('_')
                mk_sh(sample_index,chr,front,tail,ifront,itail)
main()






#JG.imputation.mergeGen.processing.chr8.141000001_146000000.vcf.gz
#JG.imputation.mergeGen.processing.chr11.119000001_124000000.gen.gz
#JG.imputation.mergeGen.processing.chr11.119000001_124000000.gen.gz
#JG.imputation.chr11.55001_60000.sampleID.129000001_134000000.gen.gz



###################################################################check

import glob,os

Dir = "05.imputation/"
inDir = Dir + "INPUTs/"
outDir = Dir + "OUTPUTs/01.imputation/"

check_list = []
####output file 을 check하는 함수
def check_region(chr,front,tail):
	static_sample = "1_5000.sampleID"
    check = outDir + "JG.imputation.%s.%s.%s_%s.gen.gz"%(chr,static_sample,front,tail)

	if glob.glob(check):
		return
	else:
		check_list.append(chr+"."+front+"_"+tail)

####chunk imputation정보리시트를 읽고, 해당 chunk가 잘 만들어 졌는지 확인 
def main():
	print("main : ...")
	ref_file = open(inDir + "imputation.POS.auto.txt","r")
	ref_list = [x.replace('\n','').split('\t') for x in ref_file]
	for ref_pos,imputation_pos in ref_list[1:]:
		chr,front,tail = imputation_pos.split('_')
		check_region(chr,front,tail)

main()

#####chunk가 없는 리스트를 .txt 파일로 저장
outfile = open(Dir+"check_Imputation_position_list.txt", "w")
outfile.write("chr.front_tail\n")

for i in check_list:
	outfile.write(i+"\n")
outfile.close()



##########################################################
######infoscore


######

import glob,os

Dir = "05.imputation/"
inDir = Dir + "INPUTs/"
outDir = Dir + "OUTPUTs/"
shDir = Dir + "SCRIPTs/03.infoscore/"

mergeDir = outDir + "02.mergeGen/"
tool ="TOOLs/qctool"

def mk_sh(gen):
	print("gen file = " + gen)
	gen_input = mergeDir + gen
	info_output = outDir+"03.info/" + gen.replace("gen.gz","") + "info"
	with open(shDir + gen.replace("gen.gz","") + "info.sh",'w') as shwrite:
		shwrite.write("%s -g %s -snp-stats -osnp %s"%(tool,gen_input,info_output))


def main():
        print("main : ...")

	gens = glob.glob(mergeDir+"*pro*gz")
	gen_index = [s.replace(mergeDir,"") for s in gens]

	for gen in gen_index:
		mk_sh(gen)
main()


